In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import model_selection, datasets, svm
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

import data_process_funcs
import meta_dataframe_functions


In [16]:
df_arr = data_process_funcs.arr_dfs_of_qubit_sizes('Hardware')
df_4q_H = df_arr[0]
df_4q_H.head()

,circuit_type,backend,nr_qubits,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,torino,4,766,1.0,0.513055,0.268930,0.067885,0.049608,0.007833,0.007833,0.060052,0.015666,0.000000,0.000000,0.000000,0.001305,0.001305,0.003916,0.002611
1,1,torino,4,740,1.0,0.495946,0.255405,0.082432,0.059459,0.016216,0.018919,0.040541,0.016216,0.002703,0.002703,0.001351,0.002703,0.000000,0.002703,0.002703
2,1,torino,4,776,1.0,0.416237,0.337629,0.083763,0.065722,0.006443,0.010309,0.052835,0.012887,0.002577,0.001289,0.000000,0.001289,0.000000,0.001289,0.007732
3,1,torino,4,799,1.0,0.468085,0.255319,0.093867,0.067584,0.011264,0.015019,0.056320,0.015019,0.001252,0.003755,0.000000,0.002503,0.000000,0.002503,0.007509
4,1,torino,4,785,1.0,0.496815,0.271338,0.082803,0.053503,0.014013,0.008917,0.045860,0.016561,0.001274,0.003822,0.000000,0.001274,0.000000,0.001274,0.002548


In [8]:
def features_to_int(df):
    df_ = df
    label_encoder = LabelEncoder()
    df_['backend'] = label_encoder.fit_transform(df_['backend'])
    df_['circuit_type'] = pd.to_numeric(df_['circuit_type'], downcast='integer', errors='coerce')
    df_ = df_.drop('nr_qubits', axis = 1)
    return df_

In [ ]:
df_4q_H = features_to_int(df_4q_H)


In [21]:

df_4q_H.head()

,circuit_type,backend,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,1,766,1.0,0.513055,0.268930,0.067885,0.049608,0.007833,0.007833,0.060052,0.015666,0.000000,0.000000,0.000000,0.001305,0.001305,0.003916,0.002611
1,1,1,740,1.0,0.495946,0.255405,0.082432,0.059459,0.016216,0.018919,0.040541,0.016216,0.002703,0.002703,0.001351,0.002703,0.000000,0.002703,0.002703
2,1,1,776,1.0,0.416237,0.337629,0.083763,0.065722,0.006443,0.010309,0.052835,0.012887,0.002577,0.001289,0.000000,0.001289,0.000000,0.001289,0.007732
3,1,1,799,1.0,0.468085,0.255319,0.093867,0.067584,0.011264,0.015019,0.056320,0.015019,0.001252,0.003755,0.000000,0.002503,0.000000,0.002503,0.007509
4,1,1,785,1.0,0.496815,0.271338,0.082803,0.053503,0.014013,0.008917,0.045860,0.016561,0.001274,0.003822,0.000000,0.001274,0.000000,0.001274,0.002548


In [22]:
df_4q_H.tail()

,circuit_type,backend,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
337,3,0,933,1.0,0.516613,0.082529,0.035370,0.088960,0.060021,0.003215,0.002144,0.116827,0.067524,0.003215,0.005359,0.008574,0.008574,0.001072,0.0
338,3,0,964,1.0,0.502075,0.082988,0.038382,0.085062,0.044606,0.004149,0.005187,0.128631,0.090249,0.003112,0.004149,0.007261,0.003112,0.001037,0.0
339,3,0,993,1.0,0.496475,0.091641,0.040282,0.078550,0.049345,0.005035,0.005035,0.132931,0.078550,0.001007,0.002014,0.007049,0.011078,0.001007,0.0
340,3,0,871,1.0,0.493685,0.090700,0.029851,0.087256,0.066590,0.005741,0.004592,0.118255,0.084960,0.002296,0.001148,0.006889,0.006889,0.001148,0.0
341,3,0,908,1.0,0.511013,0.090308,0.034141,0.067181,0.064978,0.004405,0.003304,0.128855,0.083700,0.002203,0.004405,0.003304,0.002203,0.000000,0.0


In [23]:
Y = df_4q_H[['backend']]
X = df_4q_H.drop('backend',axis = 1)

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X,Y,train_size=40,shuffle = True,random_state=42)

In [24]:
X.head()

,circuit_type,totalError,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,1,766,1.0,0.513055,0.268930,0.067885,0.049608,0.007833,0.007833,0.060052,0.015666,0.000000,0.000000,0.000000,0.001305,0.001305,0.003916,0.002611
1,1,740,1.0,0.495946,0.255405,0.082432,0.059459,0.016216,0.018919,0.040541,0.016216,0.002703,0.002703,0.001351,0.002703,0.000000,0.002703,0.002703
2,1,776,1.0,0.416237,0.337629,0.083763,0.065722,0.006443,0.010309,0.052835,0.012887,0.002577,0.001289,0.000000,0.001289,0.000000,0.001289,0.007732
3,1,799,1.0,0.468085,0.255319,0.093867,0.067584,0.011264,0.015019,0.056320,0.015019,0.001252,0.003755,0.000000,0.002503,0.000000,0.002503,0.007509
4,1,785,1.0,0.496815,0.271338,0.082803,0.053503,0.014013,0.008917,0.045860,0.016561,0.001274,0.003822,0.000000,0.001274,0.000000,0.001274,0.002548


In [7]:
def fit_and_get_val_score_SVM(X_train,Y_train,X_val,Y_val,kernel_ = 'rbf',ravel = True,degree_ = 3):
    baseline = SVC(kernel=kernel_,degree = degree_)
    if ravel:
        Y_train_1d = Y_train.to_numpy()
        Y_train_1d = Y_train_1d.ravel()
    else:
        Y_train_1d = Y_train
    baseline.fit(X_train, Y_train_1d)
    Cscore = baseline.score(X_val, Y_val)
    print("SVM Validation Accuracy:", Cscore)
    return baseline

def fit_and_get_val_score_SVM2(baseline_,X_train,Y_train,X_val,Y_val,ravel = True):
    baseline = baseline_
    if ravel:
        Y_train_1d = Y_train.to_numpy()
        Y_train_1d = Y_train_1d.ravel()
    else:
        Y_train_1d = Y_train
    baseline.fit(X_train, Y_train_1d)
    Cscore = baseline.score(X_val, Y_val)
    print("SVM Validation Accuracy:", Cscore)
    return baseline

In [30]:
fit_and_get_val_score_SVM(X_train,Y_train,X_test,Y_test)

SVM Validation Accuracy: 0.6125827814569537


SVC()

In [31]:
X_train.drop('0000',axis =1)
X_test.drop('0000',axis=1)
fit_and_get_val_score_SVM(X_train,Y_train,X_test,Y_test)

SVM Validation Accuracy: 0.6125827814569537


SVC()

In [33]:
fit_and_get_val_score_SVM(X_train,Y_train,X_test,Y_test, kernel_ = 'linear')

SVM Validation Accuracy: 0.7748344370860927


SVC(kernel='linear')

In [51]:
baseline = SVC(kernel='poly',degree = 6, decision_function_shape='ovo')
fit_and_get_val_score_SVM2(baseline,X_train,Y_train,X_test,Y_test)

SVM Validation Accuracy: 0.6390728476821192


SVC(decision_function_shape='ovo', degree=6, kernel='poly')

In [53]:
baseline = SVC(kernel='poly',degree = 6, coef0=1)
fit_and_get_val_score_SVM2(baseline,X_train,Y_train,X_test,Y_test)

SVM Validation Accuracy: 0.6456953642384106


SVC(coef0=1, degree=6, kernel='poly')

In [61]:
baseline = SVC(kernel='poly', gamma='auto')
fit_and_get_val_score_SVM2(baseline,X_train,Y_train,X_test,Y_test)

SVM Validation Accuracy: 1.0


SVC(gamma='auto', kernel='poly')

In [34]:
fit_and_get_val_score_SVM(X_train,Y_train,X_test,Y_test, kernel_ = 'poly')

SVM Validation Accuracy: 0.6291390728476821


SVC(kernel='poly')

In [35]:
fit_and_get_val_score_SVM(X_train,Y_train,X_test,Y_test, kernel_ = 'sigmoid')

SVM Validation Accuracy: 0.4966887417218543


SVC(kernel='sigmoid')

In [ ]:
df_arr = data_process_funcs.arr_dfs_of_qubit_sizes('Hardware')


In [70]:

df_8q_H = df_arr[1]

In [74]:
df_8q_H = features_to_int(df_8q_H)

In [75]:
Y = df_8q_H[['backend']]
X = df_8q_H.drop('backend',axis = 1)
X = X.drop('00000000',axis = 1)

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X,Y,train_size=40,shuffle = True,random_state=42)

In [76]:
baseline = SVC(kernel='poly', gamma='auto')
fit_and_get_val_score_SVM2(baseline,X_train,Y_train,X_test,Y_test)

SVM Validation Accuracy: 0.95


SVC(gamma='auto', kernel='poly')

In [ ]:
#df_arrS = data_process_funcs.arr_dfs_of_qubit_sizes('Simulation', nr_qubit_sizes=3,nr_machines = 5)

In [ ]:
df_4q_Sb = data_process_funcs.get_expanded_df('Simulation',4)
df_4q_Sb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 0 to 99
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   circuit_type     600 non-null    object 
 1   backend          600 non-null    object 
 2   nr_qubits        600 non-null    object 
 3   experiment_type  600 non-null    object 
 4   totalError       600 non-null    int64  
 5   0000             600 non-null    float64
 6   0001             600 non-null    float64
 7   0010             600 non-null    float64
 8   0011             600 non-null    float64
 9   0100             600 non-null    float64
 10  0101             600 non-null    float64
 11  0110             600 non-null    float64
 12  0111             600 non-null    float64
 13  1000             600 non-null    float64
 14  1001             600 non-null    float64
 15  1010             600 non-null    float64
 16  1011             600 non-null    float64
 17  1100             600 n

In [32]:
df_4q_Hb['backend'].info()

<class 'pandas.core.series.Series'>
Index: 342 entries, 0 to 56
Series name: backend
Non-Null Count  Dtype 
--------------  ----- 
342 non-null    object
dtypes: object(1)
memory usage: 5.3+ KB


In [19]:
df_4q_S = data_process_funcs.get_expanded_df('Simulation',4)
# df_8q_S = data_process_funcs.get_expanded_df('Simulation',8)
# df_16q_S = data_process_funcs.get_expanded_df('Simulation',16)

In [29]:
df_4q_S.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 99
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   circuit_type     1200 non-null   object 
 1   backend          1200 non-null   object 
 2   nr_qubits        1200 non-null   object 
 3   experiment_type  1200 non-null   object 
 4   totalError       1200 non-null   int64  
 5   0000             1199 non-null   float64
 6   0001             1199 non-null   float64
 7   0010             1199 non-null   float64
 8   0011             1199 non-null   float64
 9   0100             1199 non-null   float64
 10  0101             1199 non-null   float64
 11  0110             1199 non-null   float64
 12  0111             1199 non-null   float64
 13  1000             1199 non-null   float64
 14  1001             1199 non-null   float64
 15  1010             1199 non-null   float64
 16  1011             1199 non-null   float64
 17  1100             1199

In [5]:
def get_x_y(df_q):
    df_q = features_to_int(df_q)
    Y = df_q[['backend']]
    X = df_q.drop('backend',axis = 1)
    return X,Y


In [9]:
X,Y = get_x_y(df_4q_S)

X = X.drop('0000',axis = 1)
X = X.drop('experiment_type',axis = 1)

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X,Y,train_size=80,shuffle = True,random_state=42)


In [28]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1120 entries, 78 to 14
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   circuit_type  1120 non-null   int8   
 1   totalError    1120 non-null   int64  
 2   0001          1119 non-null   float64
 3   0010          1119 non-null   float64
 4   0011          1119 non-null   float64
 5   0100          1119 non-null   float64
 6   0101          1119 non-null   float64
 7   0110          1119 non-null   float64
 8   0111          1119 non-null   float64
 9   1000          1119 non-null   float64
 10  1001          1119 non-null   float64
 11  1010          1119 non-null   float64
 12  1011          1119 non-null   float64
 13  1100          1119 non-null   float64
 14  1101          1119 non-null   float64
 15  1110          1119 non-null   float64
 16  1111          1119 non-null   float64
dtypes: float64(15), int64(1), int8(1)
memory usage: 149.8 KB


In [10]:

baseline = SVC(kernel='poly', gamma='auto')

fit_and_get_val_score_SVM2(baseline,X_train,Y_train,X_test,Y_test)

ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values